 9. Train a classification model for the SketchRNN dataset, available in TensorFlow datasets


In [15]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from pathlib import Path

In [10]:

tf_download_root = 'http://download.tensorflow.org/data/'
filename = 'quickdraw_tutorial_dataset_v1.tar.gz'
filepath = tf.keras.utils.get_file(filename, tf_download_root + filename, cache_dir='.', extract=True)

1065301781/1065301781 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [18]:
quickdraw_dir = Path(filepath).parent
train_files = sorted([str(p) for p in (quickdraw_dir.glob('training.tfrecord-*'))])
valid_files = sorted([str(p) for p in (quickdraw_dir.glob('eval.tfrecord-*'))])

In [19]:
with open(quickdraw_dir / 'eval.tfrecord.classes') as test_classes_file:
    test_classes = test_classes_file.readlines()

with open(quickdraw_dir/'training.tfrecord.classes') as train_classes_file:
    train_classes = train_classes_file.readlines()



In [22]:
assert train_classes == test_classes
class_names = [name.strip().lower() for name in train_classes]

In [24]:
sorted(class_names)[:15]

['aircraft carrier',
 'airplane',
 'alarm clock',
 'ambulance',
 'angel',
 'animal migration',
 'ant',
 'anvil',
 'apple',
 'arm',
 'asparagus',
 'axe',
 'backpack',
 'banana',
 'bandage']

let's know the stracture of the data

In [29]:
def inspect_record(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset([tfrecord_file])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)

# Inspect a TFRecord
inspect_record('datasets/training.tfrecord-00000-of-00010')

features {
  feature {
    key: "shape"
    value {
      int64_list {
        value: 55
        value: 3
      }
    }
  }
  feature {
    key: "ink"
    value {
      float_list {
        value: 0.01968503
        value: 0.0509554148
        value: 0
        value: 0.0314961076
        value: 0.0318471193
        value: 0
        value: 0.122047246
        value: -0.0127388239
        value: 0
        value: 0.0669291
        value: -0.0509554148
        value: 0
        value: -0.015748024
        value: -0.133757979
        value: 0
        value: -0.0551181436
        value: -0.178343952
        value: 0
        value: -0.047244072
        value: -0.0445859879
        value: 0
        value: -0.043307066
        value: -0.00636942685
        value: 0
        value: -0.0590551496
        value: 0.0509554148
        value: 0
        value: -0.0905511677
        value: 0.191082805
        value: 0
        value: -0.003937006
        value: 0.114649653
        value: 0
        value: 

we can feed this stracture to any llm and ask for parsing function for easy using or extract the stracture manualy.

**Data stracture and further details about next function** in this disscussion:

https://chatgpt.com/share/6757b589-92bc-800a-8dfc-913c678f8f8d

In [ ]:
feature_descriptions = {
        "ink": tf.io.VarLenFeature(dtype=tf.float32),
        "shape": tf.io.FixedLenFeature([2], dtype=tf.int64),
        "class_index": tf.io.FixedLenFeature([1], dtype=tf.int64)
    }

def parse(data_batch):
  examples = tf.io.parse_example(data_batch, feature_descriptions)
  flat_sketches = tf.sparse.to_dense(examples['ink'])
  sketches = tf.reshape(flat_sketches, shape=[tf.size(data_batch),-1,3])#reshaping to (batchsize, number of points, the 3 is triplets x,y,t)
  lengths = examples['shape'][:,0]#selects number of points in each example over the batch
  labels = examples['class_index'][:,0]#this removes the extra dim
  return sketches, lengths, labels